In [74]:
import requests, json
import cognitive_face as CF
from pprint import pprint
from pathlib import Path
from pymongo import MongoClient

In [4]:
def getFaceAPICreds():
    credentials = json.load(open('../credentials.json'))
    faceCreds = credentials['cognitiveServices']['faceDetection']
    return faceCreds

In [5]:
def captureImages():
    # Note: The captured images must be named face**.jpeg; 
    # Where ** represents numbers ranging from 00 to 10
    # We need to get eleven images to train the model
    pprint("Images captured!")

In [93]:
def getFaceDetectURL(faceCreds):
    return(faceCreds['endPoint'] + '/detect')

def getPersonGroupURL(faceCreds):
    return(faceCreds['endPoint'] + '/persongroups')

In [8]:
def getFaceKey(faceCreds):
    return(faceCreds['key'])

In [9]:
def checkCountValue(count):
    if(count<10):
        count = "0" + str(count)
        return(count)
    else:
        return(count)

In [10]:
def renameImages(p):
    count = 0
    for file in p.glob('*.jpg'):
        imageCount = checkCountValue(count)
        newImageName = "image" + checkCountValue(count) + ".jpg"
        print("Current file name is: ")
        print(file)
        file.rename(newImageName)
        count = count + 1

In [11]:
def createHeaders(subscription_key, contentType):
    headers = {
    'Content-Type': 'application/' + contentType,
    'Ocp-Apim-Subscription-Key': subscription_key,
    }
    return(headers)

In [12]:
def createParams():
    params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,blur,exposure,noise'
    }
    return(params)

In [13]:
def getFaceDetails(res):
    faceId = res.json()['faceId']
    noiseLevel = res.json()['noise']['noiseLevel']
    blurLevel = res.json()['blur']['blurLevel']
#     all attributes not necessary; for extra functionality later:
    return({
        "faceId" : faceId,
        "noiseLevel" : noiseLevel,
        "blurLevel" : blurLevel
    })

In [14]:
def getFaceId():
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'octet-stream')
    params = createParams()
    url = getFaceDetectURL(faceCreds)
    body = open('./target.jpg', 'rb')
    response = requests.post(url=url,data=body,params=params,headers=headers)
    body.close()
    faceID = getFaceDetails(response)['faceId']
    return(faceID)

In [30]:
def createPersonGroup(groupDetails):
    faceCreds = getFaceAPICreds()
    url = getPersonGroupURL(faceCreds)
    fullURL = url + "/" + groupId
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, "json")
    groupId = groupDetails['id']
    groupName = groupDetails['name']
    userData = groupDetails['userData']
    jsonBody = {
        "name" : groupName,
        "userData" : userData
    }
    req = requests.put(url=fullURL, json=jsonBody, headers=headers)
    if(req.status_code == 200):
        print("Successfully created person group")
    else:
        print("Something went wrong with person group creation")
        print("Status code: " + req.status_code + ". Reason: " + req.reason)
    return(req)

In [25]:
groupDetails = {
    'id' : 'hospital_department',
    'name' : 'manipal_orthopedics',
    'userData' : 'group that manages Manipal hospital patients under the orthopedics department'
}

In [79]:
# res = createPersonGroup(groupDetails)

In [39]:
def getPersonGroupDetails(personGroupId):
    faceCreds = getFaceAPICreds()
    url = getPersonGroupURL(faceCreds)
    headers = createHeaders(getFaceKey(faceCreds), 'json')
    fullURL = url + "/" + personGroupId
    req = requests.get(url=fullURL, headers=headers)
    if(req.status_code == 200):
        print("Name of group: " + req['name'])
        print("Group ID of group: " + req['personGroupId'])
        print("User data of group: " + req['userData'])
    else:
        print("Error: " + str(req.status_code))
    return(req)

In [43]:
def listPersonGroups():
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    url = getPersonGroupURL(faceCreds)
    headers = createHeaders(key, 'json')
    req = requests.get(url=url, headers=headers)
    if(req.status_code == 200):
        print("Success")
    else:
        print("Something went wrong")
    return(req)

In [56]:
def getPersonGroupTrainingStatus(personGroupId):
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'json')
    url = getPersonGroupURL(faceCreds)
    fullURL = url + '/' + personGroupId + '/training'
    req = requests.get(url=fullURL, headers=headers)
    if(req.status_code != 200):
        print(req.json()['error']['message'])
    else:
        print(req.json()['status'])
    return(req)

In [58]:
def trainPersonGroup(personGroupId):
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'json')
    url = getPersonGroupURL(faceCreds)
    fullURL = url + '/' + personGroupId + '/train'
    req = requests.get(url=fullURL, headers=headers)
    if(req.status_code == 202):
        print("Training has begun")
    else:
        print("Something went wrong")
        print(req.json()['error']['message'])
    return(req)

In [89]:
def createPerson(personGroupId, name, userData):
    faceCreds = getFaceAPICreds()
    key = getFaceKey(faceCreds)
    headers = createHeaders(key, 'json')
    url = getPersonGroupURL(faceCreds)
    fullURL = url + '/' + personGroupId + '/persons'
    body = {
        "name" : name,
        "userData" : userData
    }
    req = requests.post(url=fullURL, headers=headers, json=body)
    if(req.status_code == 200):
        print("Success")
        pprint(req.json())
        dbEntryResult = savePersonId(personGroupId, name, req.json()['personId'])
        if(result.acknowledged):
            print("Stored in DB!")
    else:
        print("Error: " + str(req.status_code))
    return(req)

In [71]:
def getDBCreds():
    credentials = json.load(open('../credentials.json'))
    dbCreds = credentials['database']
    return dbCreds

def connectToDB(dbCreds):
    cosmosConnString = "mongodb://" + dbCreds['username'] + ":" + dbCreds['password'] + "@" + dbCreds['host'] + ":" + str(dbCreds['port']) + "/?ssl=true&replicaSet=globaldb"
    client = MongoClient(cosmosConnString)
    return client

def getClient():
    dbCreds = getDBCreds()
    client = connectToDB(dbCreds)
    return client

def getPatients(client):
    db = client["admin"]
    patientCollection = db["patients"]
    return patientCollection

In [84]:
def savePersonId(personGroupId, name, personId):
    client = getClient()
    patientCollection = getPatients(client)
    dbEntry = {
        "name" : name,
        "personGroupId" : personGroupId,
        "personId" : personId
    }
    result = patientCollection.insert_one(dbEntry)
    return(result)

In [99]:
# result = savePersonId("hospital_department", "chirag", "00134c21-4094-410b-91be-bf52e34f251b")

In [98]:
p = Path('./arzan')
for file in p.glob('*.jpg'):
    image = open('file', 'rb')
    addPersonFace(image, personGroupId, personId)
    

arzan\image01.jpg
arzan\image02.jpg
arzan\image03.jpg
arzan\image04.jpg
arzan\image05.jpg
arzan\image06.jpg
